In [ ]:
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [ ]:
cwd = os.getcwd()
print(cwd)
dataPath = os.path.join(cwd, 'data')
austin_housing = pd.read_csv(os.path.join(dataPath, 'crime-housing-austin-2015.csv'))
zip_codes = pd.read_csv(os.path.join(dataPath, 'AustinZipCodes.csv'))

In [ ]:
austin_housing.head()
austin_housing.columns

In [ ]:
zip_codes.head()
zip_codes.columns